In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime
from decimal import Decimal
import random
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV

In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt

## Original code

In [5]:
def run_xgb_model(t_size=0.3, n_estimators=350, max_depth=12, learning_rate=0.01, min_child_weight=5, subsample=0.7, colsample_bytree=0.6, gamma=0.2, reg_alpha=1, reg_lambda=2, k_num=10, y_var='Recharge RC 50% mm/y', y_predict='R50', aus_file='Australia_grid_0p05_data.csv', seed=42, test_data=False):
    start_time = datetime.now()
    DataLocation = os.path.join('..', 'data')
    os.chdir(DataLocation)

    df = pd.read_csv('dat07_u.csv', low_memory=False).sample(frac=1, random_state=seed)
    df.dropna(subset=['Rain mm/y', 'koppen_geiger', 'PET mm/y', 'distance_to_coast_km', 'Aridity', 'elevation_mahd', 'wtd_mbgs', 'regolith_depth_mbgs', 'slope_perc', 'clay_perc', 'silt_perc', 'sand_perc', 'soil_class', 'geology', 'ndvi_avg', 'vegex_cat', 'rainfall_seasonality'], inplace=True)
    print(f"nans removed, removed {len(df) - len(df.dropna())}, removed {Decimal(100 * (len(df) - len(df.dropna()))/len(df)).quantize(Decimal('1.0'))}%")
    print(f"Remaining data has mean Rrc/P ratio: {Decimal(np.nanmean(df['Rrc/P'])).quantize(Decimal('1.00'))}")

    train_params = ['Rain mm/y', 'rainfall_seasonality', 'PET mm/y', 'elevation_mahd', 'distance_to_coast_km', 'ndvi_avg', 'clay_perc', 'soil_class']

    aus_X = pd.read_csv(aus_file)[train_params]
    random.seed(seed)
    random_num = random.randint(0, 1000)

    if not test_data:
        X = df[train_params]
        y = df[y_var]
        Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=t_size, random_state=random_num)
    else:
        train_data_file = 'train_data.csv'
        train_data = pd.read_csv(train_data_file)
        Xtrain = train_data[train_params]
        ytrain = train_data[y_var]

    xgb = XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree, gamma=gamma, reg_alpha=reg_alpha, reg_lambda=reg_lambda, random_state=random_num)
    xgb.fit(Xtrain, ytrain)

    print(f'Training Score: {xgb.score(Xtrain, ytrain):.3f}')

    if not test_data:
        ypredv = xgb.predict(Xvalid)
        y_pred_valid = pd.DataFrame({'y_predict': ypredv, 'y_validation': yvalid})
        y_pred_valid['Residual (predicted R - CMB R)'] = y_pred_valid['y_predict'] - y_pred_valid['y_validation']
        y_pred_valid['Residual (%)'] = ((y_pred_valid['y_predict'] - y_pred_valid['y_validation']) / y_pred_valid['y_validation']) * 100
        print(f'R2 Score: {r2_score(yvalid, ypredv):.3f}')
        print(f'RMSE: {mean_squared_error(yvalid, ypredv, squared=False):.3f}')
        print(f'MAE: {mean_absolute_error(yvalid, ypredv):.3f}')

        scoring = {'r2': 'r2', 'mae': 'neg_mean_absolute_error', 'rmse': 'neg_root_mean_squared_error'}
        cv_results = cross_validate(xgb, Xtrain, ytrain, cv=k_num, scoring=scoring, n_jobs=-1)
        print(f'k={k_num}')
        print(f'R2 Score: {np.mean(cv_results["test_r2"]):.3f}')
        print(f'RMSE: {-np.mean(cv_results["test_rmse"]):.1f}')
        print(f'MAE: {-np.mean(cv_results["test_mae"]):.1f}')

        print('Starting predictions...')
        y_pred_aus = pd.DataFrame({'lat': pd.read_csv(aus_file).iloc[:,0], 'lon': pd.read_csv(aus_file).iloc[:,1], y_predict: xgb.predict(aus_X)})
        print('Finished prediction... writing values')

        y_pred_valid.to_csv(f'model_validation_predictions_errors_50_{n_estimators}n_estimators_lr{learning_rate}_{k_num}fold_out.csv', index=False)
        y_pred_aus.to_csv(f'model_predictions_aus_{n_estimators}n_estimators_lr{learning_rate}_{k_num}fold_out.csv', index=False)
    print(f'Model took: {(datetime.now() - start_time).total_seconds()/60:.2f} minutes to run')

    # Save the trained model to a file if using test data
    if test_data:
        model_dir = os.path.join('..', 'Trained_models')
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        model_filename = os.path.join(model_dir, f'xgb_model_{n_estimators}n_estimators_lr{learning_rate}.pkl')
        joblib.dump(xgb, model_filename)
        print(f'Model saved to {model_filename}')

def optimize_xgb_model(X, y, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1):
    xgb = XGBRegressor()
    grid_search = HalvingGridSearchCV(estimator=xgb, param_grid=param_grid, cv=cv, scoring=scoring, n_jobs=n_jobs, verbose=1, aggressive_elimination=True)
    grid_search.fit(X, y)
    print(f'Best parameters found: {grid_search.best_params_}')
    print(f'Best score: {grid_search.best_score_}')
    return grid_search.best_estimator_



In [ ]:
def optimize_xgb_trees(X, y, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1):
    xgb = XGBRegressor()
    grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=cv, scoring=scoring, n_jobs=n_jobs, verbose=1, aggressive_elimination=True)
    grid_search.fit(X, y)
    print(f'Best parameters found: {grid_search.best_params_}')
    print(f'Best score: {grid_search.best_score_}')
    #return grid_search.best_estimator_

    xgb_param = {
        'max_depth': 5,
        'learning_rate': 0.1,
        'min_child_weight': 1,
        'subsasmple': 0.8,
        'colsample_bytree': 0.8,
        'gamma': 0
    }
    cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=100, nfold=5,
                      metrics='auc', early_stopping_rounds=10, verbose_eval=False)
    alg.set_params(n_estimators=cvresult.shape[0])


In [7]:
if __name__ == "__main__":
    run_xgb_model(test_data=True)

nans removed, removed 0, removed 0.0%
Remaining data has mean Rrc/P ratio: 0.04
Training Score: 0.811
Model took: 0.39 minutes to run
Model saved to ..\Trained_models\xgb_model_350n_estimators_lr0.01.pkl


In [8]:
param_grid = {
    'n_estimators': [50, 75, 100, 125, 150, 200],
    'max_depth': [3, 5, 8, 10],
    'learning_rate': [0.1, 0.2, 0.3, 0.4],
    'min_child_weight': [5, 10],
    'colsample_bytree': [0.5, 0.7, 0.9, 1.],
    'gamma': [0.2]
}

df = pd.read_csv('dat07_u.csv', low_memory=False).sample(frac=1, random_state=42)
df.dropna(subset=['Rain mm/y', 'koppen_geiger', 'PET mm/y', 'distance_to_coast_km', 'Aridity', 'elevation_mahd', 'wtd_mbgs', 'regolith_depth_mbgs', 'slope_perc', 'clay_perc', 'silt_perc', 'sand_perc', 'soil_class', 'geology', 'ndvi_avg', 'vegex_cat', 'rainfall_seasonality'], inplace=True)

train_params = ['Rain mm/y', 'rainfall_seasonality', 'PET mm/y', 'elevation_mahd', 'distance_to_coast_km', 'ndvi_avg', 'clay_perc', 'soil_class']
X = df[train_params]
y = df['Recharge RC 50% mm/y']

best_model = optimize_xgb_model(X, y, param_grid)
print(f'Optimized model: {best_model}')


n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 135
max_resources_: 98568
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 1296
n_resources: 135
Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


c:\Users\nefel\miniconda3\envs\AI\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


----------
iter: 1
n_candidates: 432
n_resources: 405
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
----------
iter: 2
n_candidates: 144
n_resources: 1215
Fitting 5 folds for each of 144 candidates, totalling 720 fits
----------
iter: 3
n_candidates: 48
n_resources: 3645
Fitting 5 folds for each of 48 candidates, totalling 240 fits
----------
iter: 4
n_candidates: 16
n_resources: 10935
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 5
n_candidates: 6
n_resources: 32805
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 6
n_candidates: 2
n_resources: 98415
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best parameters found: {'colsample_bytree': 0.8, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 50, 'min_child_weight': 20, 'n_estimators': 50}
Best score: -2816.0756165242346
Optimized model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_by

In [ ]:
train_data_file = 'train_data.csv'
train_data = pd.read_csv(train_data_file)
Xtrain = train_data[train_params]
ytrain = train_data[y_var]

In [9]:
if __name__ == "__main__":
    run_xgb_model(gamma=0, n_estimators=50, max_depth=5, learning_rate=0.1, min_child_weight=1, subsample=0.8, colsample_bytree=0.8, test_data=True)

nans removed, removed 0, removed 0.0%
Remaining data has mean Rrc/P ratio: 0.04
Training Score: 0.824
Model took: 0.38 minutes to run
Model saved to ..\Trained_models\xgb_model_50n_estimators_lr0.1.pkl


## random uniform attempt

In [26]:
def run_xgb_model(t_size=0.3, n_estimators=50, max_depth=30, learning_rate=0.01, min_child_weight=20, colsample_bytree=0.85, gamma=0.2, k_num=10, y_var='Recharge RC 50% mm/y', y_predict='R50', aus_file='Australia_grid_0p05_data.csv', seed=42, test_data=False):
    start_time = datetime.now()
    DataLocation = os.path.join('..', 'data')
    os.chdir(DataLocation)

    df = pd.read_csv('dat07_u.csv', low_memory=False).sample(frac=1, random_state=seed)
    df.dropna(subset=['Rain mm/y', 'koppen_geiger', 'PET mm/y', 'distance_to_coast_km', 'Aridity', 'elevation_mahd', 'wtd_mbgs', 'regolith_depth_mbgs', 'slope_perc', 'clay_perc', 'silt_perc', 'sand_perc', 'soil_class', 'geology', 'ndvi_avg', 'vegex_cat', 'rainfall_seasonality'], inplace=True)
    print(f"nans removed, removed {len(df) - len(df.dropna())}, removed {Decimal(100 * (len(df) - len(df.dropna()))/len(df)).quantize(Decimal('1.0'))}%")
    print(f"Remaining data has mean Rrc/P ratio: {Decimal(np.nanmean(df['Rrc/P'])).quantize(Decimal('1.00'))}")

    train_params = ['Rain mm/y', 'rainfall_seasonality', 'PET mm/y', 'elevation_mahd', 'distance_to_coast_km', 'ndvi_avg', 'clay_perc', 'soil_class']

    aus_X = pd.read_csv(aus_file)[train_params]
    random.seed(seed)
    random_num = random.randint(0, 1000)

    if not test_data:
        X = df[train_params]
        y = df[y_var]
        Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=t_size, random_state=random_num)
    else:
        train_data_file = 'train_data.csv'
        train_data = pd.read_csv(train_data_file)
        Xtrain = train_data[train_params]
        ytrain = train_data[y_var]

    xgb = XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, min_child_weight=min_child_weight, colsample_bytree=colsample_bytree, gamma=gamma, random_state=random_num)
    xgb.fit(Xtrain, ytrain)

    print(f'Training Score: {xgb.score(Xtrain, ytrain):.3f}')

    if not test_data:
        ypredv = xgb.predict(Xvalid)
        y_pred_valid = pd.DataFrame({'y_predict': ypredv, 'y_validation': yvalid})
        y_pred_valid['Residual (predicted R - CMB R)'] = y_pred_valid['y_predict'] - y_pred_valid['y_validation']
        y_pred_valid['Residual (%)'] = ((y_pred_valid['y_predict'] - y_pred_valid['y_validation']) / y_pred_valid['y_validation']) * 100
        print(f'R2 Score: {r2_score(yvalid, ypredv):.3f}')
        print(f'RMSE: {mean_squared_error(yvalid, ypredv, squared=False):.3f}')
        print(f'MAE: {mean_absolute_error(yvalid, ypredv):.3f}')

        scoring = {'r2': 'r2', 'mae': 'neg_mean_absolute_error', 'rmse': 'neg_root_mean_squared_error'}
        cv_results = cross_validate(xgb, Xtrain, ytrain, cv=k_num, scoring=scoring, n_jobs=-1)
        print(f'k={k_num}')
        print(f'R2 Score: {np.mean(cv_results["test_r2"]):.3f}')
        print(f'RMSE: {-np.mean(cv_results["test_rmse"]):.1f}')
        print(f'MAE: {-np.mean(cv_results["test_mae"]):.1f}')

        print('Starting predictions...')
        y_pred_aus = pd.DataFrame({'lat': pd.read_csv(aus_file).iloc[:,0], 'lon': pd.read_csv(aus_file).iloc[:,1], y_predict: xgb.predict(aus_X)})
        print('Finished prediction... writing values')

        y_pred_valid.to_csv(f'model_validation_predictions_errors_50_{n_estimators}n_estimators_lr{learning_rate}_{k_num}fold_out.csv', index=False)
        y_pred_aus.to_csv(f'model_predictions_aus_{n_estimators}n_estimators_lr{learning_rate}_{k_num}fold_out.csv', index=False)
    print(f'Model took: {(datetime.now() - start_time).total_seconds()/60:.2f} minutes to run')

    # Save the trained model to a file if using test data
    if test_data:
        model_dir = os.path.join('..', 'Trained_models')
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        model_filename = os.path.join(model_dir, f'xgb_model_{n_estimators}n_estimators_lr{learning_rate}.pkl')
        joblib.dump(xgb, model_filename)
        print(f'Model saved to {model_filename}')

from sklearn.model_selection import RandomizedSearchCV


def optimize_xgb_model(X, y, param_distributions, n_iter=100, cv=5, scoring='neg_mean_squared_error', n_jobs=-1):
    xgb = XGBRegressor()
    random_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=cv,
        scoring=scoring,
        n_jobs=n_jobs,
        verbose=1,
        random_state=42
    )
    random_search.fit(X, y)
    print(f'Best parameters found: {random_search.best_params_}')
    print(f'Best score: {random_search.best_score_}')
    return random_search.best_estimator_

# Define the parameter distributions
param_distributions = {
    'colsample_bytree': np.random.uniform(0.8, 1.0, 10),
    'gamma': np.random.uniform(0.2, 0.6, 10),
    'learning_rate': np.random.uniform(0.1, 0.5, 10),
    'max_depth': np.random.randint(10, 111, 10),
    'min_child_weight': np.random.randint(10, 31, 10),
    'n_estimators': np.random.randint(50, 201, 10)
}

In [27]:
df = pd.read_csv('dat07_u.csv', low_memory=False).sample(frac=1, random_state=42)
df.dropna(subset=['Rain mm/y', 'koppen_geiger', 'PET mm/y', 'distance_to_coast_km', 'Aridity', 'elevation_mahd', 'wtd_mbgs', 'regolith_depth_mbgs', 'slope_perc', 'clay_perc', 'silt_perc', 'sand_perc', 'soil_class', 'geology', 'ndvi_avg', 'vegex_cat', 'rainfall_seasonality'], inplace=True)

train_params = ['Rain mm/y', 'rainfall_seasonality', 'PET mm/y', 'elevation_mahd', 'distance_to_coast_km', 'ndvi_avg', 'clay_perc', 'soil_class']
X = df[train_params]
y = df['Recharge RC 50% mm/y']

best_model = optimize_xgb_model(X, y, param_distributions)
print(f'Optimized model: {best_model}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [23]:
if __name__ == "__main__":
    run_xgb_model(test_data=True)

nans removed, removed 0, removed 0.0%
Remaining data has mean Rrc/P ratio: 0.04
Training Score: 0.469
Model took: 0.11 minutes to run
Model saved to ..\Trained_models\xgb_model_50n_estimators_lr0.01.pkl


Best parameters found: {'colsample_bytree': 0.9, 'gamma': 0.2, 'learning_rate': 0.15, 'max_depth': 10, 'min_child_weight': 20, 'n_estimators': 100}
Best score: -2832.4503219329135
Optimized model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.2, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=20, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

nans removed, removed 0, removed 0.0%
Remaining data has mean Rrc/P ratio: 0.04
Training Score: 0.823
Model took: 0.13 minutes to run
Model saved to ..\Trained_models\xgb_model_100n_estimators_lr0.15.pkl

Best parameters found: {'colsample_bytree': 0.85, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 30, 'min_child_weight': 20, 'n_estimators': 50}
Best score: -2815.121360563002
Optimized model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.85, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.2, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=30, max_leaves=None,
             min_child_weight=20, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)


nans removed, removed 0, removed 0.0%
Remaining data has mean Rrc/P ratio: 0.04
Training Score: 0.848
Model took: 0.15 minutes to run
Model saved to ..\Trained_models\xgb_model_50n_estimators_lr0.1.pkl